Basic RNN for text modeling

In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR) #disable warnings
import numpy as np
import collections

In [2]:
# Hyperparameters
batchSz = 100
windowSz = 100
rnnSz = 50
embedSz = 30
vocabSz = 10000

In [3]:
# Read data
path = '/Users/cytology/simple-examples/data/ptb.train.txt'
with tf.gfile.GFile(path) as f:
    data = f.read().replace("/n", "<eos>").split()

In [4]:
batch_len = (len(data) - 1) // batchSz
epoch_size = batch_len // windowSz

In [5]:
# Convert words to IDs
counter = sorted(collections.Counter(data).items(), 
                 key=lambda x: -x[1])[:vocabSz] # sort by frequency
words, _ = list(zip(*counter))
word_to_ID = dict(zip(words, range(len(words))))
data = [word_to_ID[word] for word in data if word in word_to_ID]

In [6]:
raw_data = tf.convert_to_tensor(data, tf.int32)
batch_data = tf.reshape(raw_data[:batchSz * batch_len], 
                        [batchSz, batch_len])

In [7]:
batchX = tf.placeholder(tf.int32, [batchSz, windowSz])
batchY = tf.placeholder(tf.int32, [batchSz, windowSz])

In [8]:
E = tf.Variable(tf.random_normal([vocabSz, embedSz], stddev=0.1))

# embeddings = tf.nn.embedding_lookup(E, batch_data)



In [9]:
W = tf.Variable(tf.random_normal([rnnSz, vocabSz], stddev=0.1))
b = tf.Variable(tf.random_normal([vocabSz]))

embeddings = tf.nn.embedding_lookup(E, batchX)

rnn = tf.contrib.rnn.BasicRNNCell(rnnSz)
initialState = rnn.zero_state(batchSz, tf.float32)
outputs, nextState = tf.nn.dynamic_rnn(rnn, embeddings, initial_state=initialState)

output2 = tf.reshape(outputs, [batchSz*windowSz, rnnSz])

logits = tf.matmul(output2, W) + b

# xEnt = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=batchY)
# loss = tf.reduce_mean(xEnt)
# perplexity = tf.math.exp(loss)

# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

sess = tf.Session()
sess.run(tf.global_variables_initializer())

inputSt = sess.run(initialState)
for i in range(epoch_size):
    start = i * windowSz
    end = start + windowSz
    inpt = batch_data[:, start:end]
    targets = batch_data[:, start+1:end+1]
    logts, nxts = sess.run([logits, nextState], feed_dict={
        batchX:inpt, nextState:inputSt, batchY:targets})
    inputSt = nxts


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("strided_slice_1:0", shape=(100, 100), dtype=int32) which was passed to the feed with key Tensor("Placeholder:0", shape=(100, 100), dtype=int32).

In [ ]:
sess.run(tf.shape(logits))